## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

training_data = []
training_label = []
validation_cate_label = []
validation_data = []
validation_label = []

for train_index, test_index in kf.split(trainData):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))
    training_data.append(trainData[train_index])
    training_label.append(trainLabel[train_index])
    validation_data.append(trainData[test_index])
    validation_label.append(trainLabel[test_index])
    validation_cate_label.append(trainL[test_index])

training_data = np.array(training_data)
training_label = np.array(training_label)
validation_data = np.array(validation_data)
validation_label = np.array(validation_label)
validation_cate_label = np.array(validation_cate_label)


KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    1     2     3 ... 10147 10149 10150] len 8120 test: [    0     5    15 ... 10144 10145 10148] len 2031
trian: [    0     2     3 ... 10148 10149 10150] len 8121 test: [    1    41    43 ... 10127 10138 10147] len 2030
trian: [    0     1     2 ... 10147 10148 10149] len 8121 test: [    3     4     7 ... 10137 10141 10150] len 2030
trian: [    0     1     3 ... 10148 10149 10150] len 8121 test: [    2     9    10 ... 10139 10140 10146] len 2030
trian: [    0     1     2 ... 10147 10148 10150] len 8121 test: [    6     8    11 ... 10142 10143 10149] len 2030


In [3]:
training_data[0].shape

(8120, 9000, 1)

In [4]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def create_model():
    model = Sequential() 
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(4, activation = "softmax"))

    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate=0.001):
    
    Scores = []
    for X_train, Y_train, X_val, Y_val, val_cat in zip(training_data, training_label, validation_data, validation_label, validation_cate_label):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=20)
        train_history_1 = model.fit(x = X_train, 
                                    y = Y_train,
                                    epochs=80,
                                    validation_data=(X_val, Y_val),
                                    callbacks=[early_stop],
                                    batch_size=batch_S, 
                                    verbose=1)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(X_val, batch_size=batch_S)
        score = f1_score(val_cat, validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [60, 61, 62, 63, 64, 65, 66, 67, 68, 69]

for bat in batch_S:
    hyperparameter_index.append([bat])
    run_model(bat)

print('Finish training!')

Train on 8120 samples, validate on 2031 samples
Epoch 1/80
8120/8120 [==============================] - 15s 2ms/step - loss: 1.0138 - acc: 0.5755 - val_loss: 0.8852 - val_acc: 0.5908
Epoch 2/80
8120/8120 [==============================] - 8s 986us/step - loss: 0.8688 - acc: 0.6182 - val_loss: 0.9676 - val_acc: 0.6578
Epoch 3/80
8120/8120 [==============================] - 8s 976us/step - loss: 0.7604 - acc: 0.6767 - val_loss: 0.6815 - val_acc: 0.7179
Epoch 4/80
8120/8120 [==============================] - 8s 952us/step - loss: 0.7070 - acc: 0.7053 - val_loss: 0.6686 - val_acc: 0.7208
Epoch 5/80
8120/8120 [==============================] - 8s 987us/step - loss: 0.6731 - acc: 0.7281 - val_loss: 0.6083 - val_acc: 0.7907
Epoch 6/80
8120/8120 [==============================] - 8s 966us/step - loss: 0.6344 - acc: 0.7470 - val_loss: 0.5356 - val_acc: 0.8045
Epoch 7/80
8120/8120 [==============================] - 8s 986us/step - loss: 0.5949 - acc: 0.7663 - val_loss: 0.5398 - val_acc: 0.7937
E

8121/8121 [==============================] - 8s 988us/step - loss: 0.6749 - acc: 0.7258 - val_loss: 0.6171 - val_acc: 0.7552
Epoch 5/80
8121/8121 [==============================] - 8s 988us/step - loss: 0.6392 - acc: 0.7458 - val_loss: 0.5736 - val_acc: 0.7857
Epoch 6/80
8121/8121 [==============================] - 8s 963us/step - loss: 0.6031 - acc: 0.7607 - val_loss: 0.5921 - val_acc: 0.7754
Epoch 7/80
8121/8121 [==============================] - 8s 983us/step - loss: 0.5913 - acc: 0.7697 - val_loss: 0.5475 - val_acc: 0.7931
Epoch 8/80
8121/8121 [==============================] - 8s 993us/step - loss: 0.5668 - acc: 0.7790 - val_loss: 0.5224 - val_acc: 0.8108
Epoch 9/80
8121/8121 [==============================] - 8s 977us/step - loss: 0.5544 - acc: 0.7856 - val_loss: 0.4719 - val_acc: 0.8365
Epoch 10/80
8121/8121 [==============================] - 8s 978us/step - loss: 0.5438 - acc: 0.7934 - val_loss: 0.5139 - val_acc: 0.8108
Epoch 11/80
8121/8121 [==============================] - 8

8121/8121 [==============================] - 8s 1ms/step - loss: 0.7817 - acc: 0.6631 - val_loss: 0.6938 - val_acc: 0.6961
Epoch 4/80
8121/8121 [==============================] - 8s 994us/step - loss: 0.6950 - acc: 0.7051 - val_loss: 0.6938 - val_acc: 0.7498
Epoch 5/80
8121/8121 [==============================] - 8s 997us/step - loss: 0.6407 - acc: 0.7481 - val_loss: 0.5972 - val_acc: 0.7606
Epoch 6/80
8121/8121 [==============================] - 8s 988us/step - loss: 0.5861 - acc: 0.7742 - val_loss: 0.5555 - val_acc: 0.7862
Epoch 7/80
8121/8121 [==============================] - 8s 984us/step - loss: 0.5591 - acc: 0.7872 - val_loss: 0.5427 - val_acc: 0.7926
Epoch 8/80
8121/8121 [==============================] - 8s 976us/step - loss: 0.5243 - acc: 0.8001 - val_loss: 0.5373 - val_acc: 0.7926
Epoch 9/80
8121/8121 [==============================] - 8s 978us/step - loss: 0.5146 - acc: 0.8072 - val_loss: 0.4908 - val_acc: 0.8123
Epoch 10/80
8121/8121 [==============================] - 8s 9

Epoch 3/80
8121/8121 [==============================] - 8s 962us/step - loss: 0.7862 - acc: 0.6612 - val_loss: 0.7410 - val_acc: 0.6793
Epoch 4/80
8121/8121 [==============================] - 8s 998us/step - loss: 0.7054 - acc: 0.7041 - val_loss: 0.6742 - val_acc: 0.7369
Epoch 5/80
8121/8121 [==============================] - 8s 976us/step - loss: 0.6548 - acc: 0.7313 - val_loss: 0.6300 - val_acc: 0.7463
Epoch 6/80
8121/8121 [==============================] - 8s 989us/step - loss: 0.6095 - acc: 0.7515 - val_loss: 0.5789 - val_acc: 0.7931
Epoch 7/80
8121/8121 [==============================] - 8s 981us/step - loss: 0.5827 - acc: 0.7648 - val_loss: 0.5279 - val_acc: 0.8049
Epoch 8/80
8121/8121 [==============================] - 8s 993us/step - loss: 0.5586 - acc: 0.7784 - val_loss: 0.5094 - val_acc: 0.8118
Epoch 9/80
8121/8121 [==============================] - 8s 977us/step - loss: 0.5281 - acc: 0.7971 - val_loss: 0.4633 - val_acc: 0.8350
Epoch 10/80
8121/8121 [=========================

Epoch 10/80
8121/8121 [==============================] - 8s 958us/step - loss: 0.4985 - acc: 0.8122 - val_loss: 0.5362 - val_acc: 0.7813
Epoch 11/80
8121/8121 [==============================] - 8s 981us/step - loss: 0.4833 - acc: 0.8165 - val_loss: 0.5320 - val_acc: 0.7892
Epoch 12/80
8121/8121 [==============================] - 8s 976us/step - loss: 0.4676 - acc: 0.8207 - val_loss: 0.5291 - val_acc: 0.7941
Epoch 13/80
8121/8121 [==============================] - 8s 955us/step - loss: 0.4725 - acc: 0.8251 - val_loss: 0.5467 - val_acc: 0.7877
Epoch 14/80
8121/8121 [==============================] - 8s 955us/step - loss: 0.4583 - acc: 0.8281 - val_loss: 0.5597 - val_acc: 0.7857
Epoch 15/80
8121/8121 [==============================] - 8s 975us/step - loss: 0.4424 - acc: 0.8375 - val_loss: 0.5181 - val_acc: 0.8128
Epoch 16/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.4448 - acc: 0.8368 - val_loss: 0.5243 - val_acc: 0.7901
Epoch 17/80
8121/8121 [====================

...

Epoch 29/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3769 - acc: 0.8633 - val_loss: 0.5115 - val_acc: 0.8192
Epoch 30/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3770 - acc: 0.8616 - val_loss: 0.5050 - val_acc: 0.8222
Epoch 31/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3700 - acc: 0.8700 - val_loss: 0.4992 - val_acc: 0.8212
Epoch 32/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3611 - acc: 0.8689 - val_loss: 0.5300 - val_acc: 0.8163
Epoch 33/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3516 - acc: 0.8739 - val_loss: 0.5084 - val_acc: 0.8251
Epoch 34/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3554 - acc: 0.8726 - val_loss: 0.5093 - val_acc: 0.8246
Epoch 35/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3455 - acc: 0.8743 - val_loss: 0.5149 - val_acc: 0.8133
Epoch 36/80
8121/8121 [==============================] 

8120/8120 [==============================] - 8s 1ms/step - loss: 0.3067 - acc: 0.8853 - val_loss: 0.4052 - val_acc: 0.8562
Epoch 45/80
8120/8120 [==============================] - 8s 1ms/step - loss: 0.3071 - acc: 0.8909 - val_loss: 0.4153 - val_acc: 0.8562
Epoch 46/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.3008 - acc: 0.8868 - val_loss: 0.4019 - val_acc: 0.8631
Epoch 47/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.3170 - acc: 0.8836 - val_loss: 0.4466 - val_acc: 0.8405
Epoch 48/80
2031/2031 [==============================] - 1s 433us/step
Loss: 0.405, Accuracy: 0.860
Train on 8121 samples, validate on 2030 samples
Epoch 1/80
8121/8121 [==============================] - 17s 2ms/step - loss: 0.9942 - acc: 0.5801 - val_loss: 0.8835 - val_acc: 0.5833
Epoch 2/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.8643 - acc: 0.6173 - val_loss: 0.8293 - val_acc: 0.6360
Epoch 3/80
8121/8121 [==============================] -

Epoch 56/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.2834 - acc: 0.8969 - val_loss: 0.5033 - val_acc: 0.8581
Epoch 57/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.2810 - acc: 0.8947 - val_loss: 0.4515 - val_acc: 0.8547
Epoch 58/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.2850 - acc: 0.8959 - val_loss: 0.4873 - val_acc: 0.8493
Epoch 59/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.2791 - acc: 0.9012 - val_loss: 0.4795 - val_acc: 0.8507
Epoch 60/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.2897 - acc: 0.8955 - val_loss: 0.4488 - val_acc: 0.8631
Epoch 61/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.2847 - acc: 0.8958 - val_loss: 0.4877 - val_acc: 0.8537
Epoch 62/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.2779 - acc: 0.8993 - val_loss: 0.4561 - val_acc: 0.8586
Epoch 63/80
8121/8121 [==============================] 

8121/8121 [==============================] - 9s 1ms/step - loss: 0.3183 - acc: 0.8859 - val_loss: 0.4878 - val_acc: 0.8409
Epoch 50/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3203 - acc: 0.8857 - val_loss: 0.4732 - val_acc: 0.8384
Epoch 51/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3130 - acc: 0.8820 - val_loss: 0.4648 - val_acc: 0.8414
Epoch 52/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3120 - acc: 0.8892 - val_loss: 0.4844 - val_acc: 0.8379
Epoch 53/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3186 - acc: 0.8883 - val_loss: 0.4708 - val_acc: 0.8399
Epoch 54/80
2030/2030 [==============================] - 1s 429us/step
Loss: 0.486, Accuracy: 0.842
Train on 8121 samples, validate on 2030 samples
Epoch 1/80
8121/8121 [==============================] - 17s 2ms/step - loss: 0.9959 - acc: 0.5763 - val_loss: 0.9213 - val_acc: 0.5842
Epoch 2/80
8121/8121 [==============================] 

Epoch 55/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3087 - acc: 0.8889 - val_loss: 0.4829 - val_acc: 0.8399
Epoch 56/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3043 - acc: 0.8902 - val_loss: 0.4356 - val_acc: 0.8522
Epoch 57/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.2996 - acc: 0.8898 - val_loss: 0.5145 - val_acc: 0.8394
Epoch 58/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.3088 - acc: 0.8856 - val_loss: 0.4216 - val_acc: 0.8522
Epoch 59/80
2030/2030 [==============================] - 1s 413us/step
Loss: 0.448, Accuracy: 0.844
Train on 8121 samples, validate on 2030 samples
Epoch 1/80
8121/8121 [==============================] - 17s 2ms/step - loss: 0.9915 - acc: 0.5808 - val_loss: 0.8955 - val_acc: 0.5833
Epoch 2/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.8858 - acc: 0.6095 - val_loss: 0.8116 - val_acc: 0.6369
Epoch 3/80
8121/8121 [=====================

In [5]:
# for bat, lr in itertools.product(batch_S, learning_rate):
#     print(bat, lr)

In [6]:
import pprint
pprint.pprint(F1_result)

[array([0.79299521, 0.61499518, 0.90293158, 0.74692488]),
 array([0.77892938, 0.59071858, 0.90406739, 0.74399328]),
 array([0.77093645, 0.62751673, 0.90282558, 0.74847918]),
 array([0.77871856, 0.61419682, 0.90698912, 0.75101153]),
 array([0.78602617, 0.56754853, 0.90034178, 0.741538  ]),
 array([0.79107485, 0.63518751, 0.90477178, 0.7519926 ]),
 array([0.79359064, 0.5788767 , 0.90294437, 0.75945623]),
 array([0.77796656, 0.60595975, 0.90421467, 0.75252474]),
 array([0.79002094, 0.64131457, 0.90200091, 0.74803485]),
 array([0.79983464, 0.59102174, 0.90533305, 0.75510874])]


In [7]:
# array([0.79250755, 0.62614335, 0.90266635, 0.74891516])